# MAP (programmation fonctionnelle en ARM)

Comment faire de la programmation fonctionnelle en ARM ?

Prenons l'exemple de map :

```
map(t_in,n,t_out,f) :
  pour i = 0..n-1 :
    t_out[i] = f(t_in[i])
```

Pour traduire la boucle, nous utiliserons le registre R6 :
```
init:    mov R6,#0
boucle:  cmp R6, n? @1
         beq fin
         @ corps de la boucle
         add R6, R6, #1
         b boucle 
fin:
```

Pour obtenir n et traduire le corps de la boucle il faut utiliser les paramètres de map. 

La méthode utilisée pour le passage des paramètres est une convention de passage par registres avec le premier paramètre dans R0, puis R1, etc.  :
* R0 pour t_in
* R1 pour n
* R2 pour t_out
* R3 pour f

Pour n (@1), on aura donc R1.

Pour l'accès à t[i], comme le tableau est un tableau d'entiers codés sur 1 octet, les adresses seront R0+R6 (ou R2+R6).

Pour l'appel à f, la convention d'appel utilise aussi les registres :
* R3 pour le paramètre
* R4 pour le résultat

Il y a un conflit d'utilisation pour R3 (adresse de f pour map, et paramètre de f en même temps). Pour résoudre ce conflit, on choisit de recopier l'adresse de f dans R7 (dans map).

Le corps de la boucle se traduit donc :
```
  @t_out[i] = f(t_in[i])
  ldrb R3,[R0,R6]
  blx R7
  strb R4,[R2,R6]
```

Le code complet (avec les prologues/épilogues nécessaires pour conserver les registres temporaires et les adresses de retour, nécessaire car appel imbriqué)

In [25]:
%%writefile map.s
         .text
         .global  map
map:     push {lr}
         push {R3,R4,R6,R7}
         mov R7, R3
init:    mov R6,#0
boucle:  cmp R6, R1
         beq fin
         ldrb R3,[R0,R6]
         blx R7
         strb R4,[R2,R6]
         add R6, R6, #1
         b boucle 
fin:
         pop {R3,R4,R6,R7}
         pop {lr}
         bx lr

Overwriting map.s


On peut vérifier que cela compile, mais pas encore que cela s'exécute :

In [10]:
%%sh
arm-linux-gnueabi-gcc -static -c map.s

Pour exécuter, il faut aussi traduire l'appel à map, avec une fonction.

La fonction est donnée dans fg.s :

In [8]:
%%writefile fg.s
         .global plus_un, fois_deux
         .text
plus_un: add      r4, r3, #1
         bx lr
fois_deux: mov      r4, r3, LSL #1
         bx lr

Overwriting fg.s


Pour l'appel :
```
  ldr R0,LD_t_in
  mov R1,#10
  ldr R2, lD_t_out
  ldr R3, plus_un
  bl map
```

On peut donc maintenant lancer la compilation complète et exécuter :

In [27]:
%%sh 
arm-linux-gnueabi-gcc -static -c map.s
arm-linux-gnueabi-gcc -static -c fg.s
arm-linux-gnueabi-gcc -static -c essai-map.s
arm-linux-gnueabi-gcc -static -c gestion_tab.s
arm-linux-gnueabi-gcc -static -c es.s
arm-linux-gnueabi-gcc -static map.o fg.o essai-map.o gestion_tab.o es.o -o essai-map
echo "0 1 2 3 4 5 6 7 8 9" | qemu-arm essai-map

Saisir une sequence de 10 entiers :
 
Sequence donnee S :
0 1 2 3 4 5 6 7 8 9  
 
map(S, plus_un) :
1 2 3 4 5 6 7 8 9 10  
